In [ ]:
%pip install pandas
%pip install transformers
%pip install accelerate

In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [2]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
import pandas as pd
df = pd.read_csv('test_subset.csv')


In [ ]:
#text set for getting the results from LLM
texts = list(df.text)

### Chat propmt used to get the labels of sequences using the fine-tuned model

In [5]:
load_in_4bit = True
dtype = None
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "Hina541/fine-tuned-gemmaModelSequence_latest_20Epochs", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 512,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)
prompt = f"Classify the sentiment of each text into ONE of the three classes: neutral, negative or positive. Text:"

answers = []
#text_list =[ "Ukraine deckt auf 😂😂😂😂😂\nDie mussten was sagen, nicht das man denkt der Staat wäre korrupt und es würde keinen Kampf dagegen geben ☝️\n\nKorrupt bleibt korrupt - auch die, da wird sich auch nichts änder 😆",'Nur Politiker wollen Krieg ! Was wenn keiner mehr Bock hat für diese 5 Politiker zu sterben warum drehen wir den Spieß nich einfach um ? 😕 frage für n Freund']
for text in texts:
    chat_template = [{"role" : "user", "content" : f"{prompt} {text}"}]

    chat_prompt = tokenizer.apply_chat_template(chat_template, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer.encode(chat_prompt, add_special_tokens=False, return_tensors="pt", max_length=512)
    outputs = model.generate(input_ids=inputs.to("cuda"), max_new_tokens = 512)
    answer = tokenizer.decode(outputs[0])
    answers.append(answer)
    #print(answer)
print(len(answers))


/ltstorage/home/1samreen/miniconda3/envs/new_saint_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Gemma patching. Transformers = 4.44.0.
   \\   /|    GPU: NVIDIA A100 80GB PCIe. Max memory: 79.138 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
Unsloth 2024.8 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


300


### Pre-processing the results in the desired format , so as to evaluate the accuracy

In [7]:
labels = []
for answer in answers:
    #print(answer.split('<start_of_turn>model')[1].strip()[:-5])
    labels.append(answer.split('<start_of_turn>model')[1].strip()[:-5])
print(len(labels))

300


In [8]:
df['predicted_sentiment_llm'] = labels

In [9]:
#getting only the classes 'Negative','Positive,'and 'Neutral' and ignoring other classes if any other are present
df = df[('negative' == df['predicted_sentiment_llm']) |  ('positive' == df['predicted_sentiment_llm']) | ('neutral' == df['predicted_sentiment_llm'])]
df = df.reset_index(drop=True)
for i in range(len(df)):
    df.loc[i,'predicted_sentiment_llm'] = str(df['predicted_sentiment_llm'][i]).lower()

In [ ]:
correct_predictions = 0
for i in range(len(df)):
    if df['sentiment'][i] == df['predicted_sentiment_llm'][i]:
        correct_predictions += 1
print('correct predictions:',correct_predictions)

In [14]:
Accuracy = (correct_predictions/len(df))*100
print('Accuracy of google gemma before fine tuning is',Accuracy, '%')

Accuracy of google gemma before fine tuning is 82.0 %


In [15]:
df.to_csv('ResultsAfterFineTuning_SequnceClassification_gemmaGoogle20Epochs.csv', index=False)